# Linear Algebra and Machine Learning Practice (Day 3)

## 목차 (Table of Contents)

**준비 과정**
- [라이브러리 설치 및 불러오기](#준비하기-라이브러리-설치-및-불러오기)
- [MNIST 데이터셋 불러오기](#MNIST-데이터셋-불러오기)

**실습 (Practice)**
1.  [행렬 분해와 선형 시스템](#1-행렬-분해와-선형-시스템-matrix-decomposition--linear-systems)
2.  [주성분 분석 (PCA)](#2-주성분-분석-principal-component-analysis-pca)

## 준비하기: 라이브러리 설치 및 불러오기

In [ ]:
# 라이브러리 설치
import subprocess
import sys

def install_if_not_exists(package):
    try:
        __import__(package)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# 모든 실습에 필요한 라이브러리 목록
required_packages = ["numpy", "matplotlib", "scipy", "scikit-learn"]
for package in required_packages:
    install_if_not_exists(package)


# 라이브러리 불러오기
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg
from sklearn.datasets import fetch_openml

# 전체 실습의 재현성을 위해 랜덤 시드를 고정합니다.
np.random.seed(0)

## MNIST 데이터셋 불러오기
scikit-learn의 fetch_openml을 사용하여 MNIST 손글씨 숫자 데이터셋을 불러옵니다.
데이터는 784개의 픽셀(28x28)로 구성된 이미지이며, 0~255 값을 갖습니다.
PCA 실습에서 사용할 수 있도록 255로 나누어 정규화합니다.

In [ ]:
print("MNIST 데이터셋을 불러오는 중... (몇 분 정도 소요될 수 있습니다)")
try:
    # as_frame=False : numpy array로 받기
    # parser='auto' : 최신 scikit-learn에서 권장하는 파서
    mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
    
    X_mnist_data = mnist.data / 255.0  # 정규화
    y_mnist_data = mnist.target.astype(int)
    
    print("MNIST 데이터셋 로드 완료.")
    print(f"데이터 형태: {X_mnist_data.shape}")
    print(f"레이블 형태: {y_mnist_data.shape}")
except Exception as e:
    print(f"데이터셋 로드 중 오류 발생: {e}")
    print("인터넷 연결을 확인하거나, 잠시 후 다시 시도해주세요.")
    X_mnist_data, y_mnist_data = None, None

---
# 1. 행렬 분해와 선형 시스템 (Matrix Decomposition & Linear Systems)
---
선형 시스템 `Ax=b`를 푸는 것은 과학과 공학의 여러 분야에서 가장 근본적인 문제 중 하나입니다.
본 실습에서는 행렬 분해(Matrix Decomposition)가 어떻게 이러한 시스템을 효율적이고 안정적으로 해결하는지 알아봅니다.

크게 두 가지 시나리오를 다룹니다:
1.  **해가 유일하게 존재하는 경우**: Cholesky 분해를 사용하여 정확한 해를 구합니다.
2.  **해가 존재하지 않는 경우**: QR 분해를 사용하여 오차를 최소화하는 최적의 근사해(최소제곱 해)를 구합니다.

## 1.1. Cholesky 분해: 정확한 해 구하기
선형 시스템 `Ax=b`에서 행렬 `A`가 **대칭(Symmetric)**이면서 **양의 정부호(Positive Definite)**인 특별한 조건을 만족할 때, Cholesky 분해를 사용하면 매우 효율적으로 해 `x`를 구할 수 있습니다.

**원리**:
- 행렬 `A`를 하삼각행렬(Lower-triangular matrix) `L`과 그 전치 행렬 `L.T`의 곱으로 분해합니다: `A = L @ L.T`
- 이를 원래 시스템에 대입하면 `(L @ L.T) @ x = b`가 되며, 문제는 두 개의 더 간단한 시스템으로 나뉩니다.
  1. `L @ y = b` 를 풀어 중간 벡터 `y`를 구합니다. (전방 대입법)
  2. `L.T @ x = y` 를 풀어 최종 해 `x`를 구합니다. (후방 대입법)

이 방식은 역행렬을 직접 구하는 것보다 수치적으로 훨씬 안정적이고 계산 비용이 저렴합니다. `scipy.linalg.cholesky`와 `scipy.linalg.solve_triangular`를 사용하여 이 과정을 실습합니다.

In [ ]:
# 실습: 아래 행렬 A와 벡터 b의 값을 직접 바꿔보며(A는 대칭 유지), 해가 어떻게 변하는지 확인해보세요.
#      A가 양의 정부호가 아니게 되면(예: A[0,0] = -1) 어떤 오류가 발생하는지도 관찰해보세요.
A_cholesky = np.array([
    [4., 2., 0.],
    [2., 5., 2.],
    [0., 2., 5.]
])
# 이 시스템의 해는 x = [1, 2, 3] 입니다. b는 A @ x의 결과입니다.
b_cholesky = np.array([8., 18., 19.])

print("행렬 A (대칭, 양의 정부호):\n", A_cholesky)
print("\n벡터 b:\n", b_cholesky)

# Cholesky 분해: A = L @ L.T
# scipy.linalg.cholesky는 기본적으로 하삼각행렬 L을 반환합니다.
try:
    L = scipy.linalg.cholesky(A_cholesky, lower=True)
    print("\n하삼각행렬 L:\n", np.round(L, 2))

    # 첫 번째 시스템 풀이: L @ y = b
    y = scipy.linalg.solve_triangular(L, b_cholesky, lower=True)
    print("\n중간 해 y:\n", np.round(y, 2))

    # 두 번째 시스템 풀이: L.T @ x = y
    x_cholesky = scipy.linalg.solve_triangular(L.T, y, lower=False)
    print("\n최종 해 x (Cholesky):\n", np.round(x_cholesky, 2))

    # 검증: np.linalg.solve와 결과 비교
    x_direct = np.linalg.solve(A_cholesky, b_cholesky)
    print("\n최종 해 x (np.linalg.solve):\n", np.round(x_direct, 2))
    print("\n두 방법의 해가 일치하는가?", np.allclose(x_cholesky, x_direct))

except np.linalg.LinAlgError:
    print("오류: 행렬이 대칭, 양의 정부호가 아닙니다.")

## 1.2. QR 분해: 최소제곱 근사해 구하기
데이터 분석에서 마주하는 많은 문제는 `Ax=b` 형태에서 행렬 `A`의 행의 개수가 열의 개수보다 많은 **Over-determined system**입니다.
이러한 시스템은 일반적으로 정확한 해가 존재하지 않습니다. 대신, 우리는 잔차(residual)의 제곱합 `||Ax - b||²`를 최소화하는 **근사해(approximate solution)** `x̂`를 찾고자 합니다. 이를 **최소제곱(Least Squares) 문제**라고 합니다.

**원리**:
- QR 분해는 행렬 `A`를 직교행렬(Orthogonal matrix) `Q`와 상삼각행렬(Upper-triangular matrix) `R`의 곱으로 표현합니다: `A = Q @ R`
- 이를 최소제곱 문제에 적용하면, 문제는 `R @ x̂ = Q.T @ b` 라는 간단한 선형 시스템으로 변환됩니다.
- `R`이 상삼각행렬이므로, 이 시스템은 후방 대입법으로 쉽게 풀 수 있습니다.

**응용 예제: 2D 데이터 직선 피팅**
산점도 데이터를 가장 잘 설명하는 직선 `y = c₀ + c₁x`를 찾는 것은 대표적인 최소제곱 문제입니다.
`N`개의 데이터 포인트 `(xᵢ, yᵢ)`가 주어졌을 때, 우리는 다음 식의 오차를 최소화하는 파라미터 `c = [c₀, c₁]`를 찾아야 합니다.

`|| A @ c - y ||²`

여기서 `A`는 첫 번째 열이 모두 1, 두 번째 열이 x 데이터로 구성된 **설계 행렬(Design Matrix)** 입니다.

In [ ]:
# 실습: true_c0, true_c1 값을 바꾸거나, 노이즈의 강도(아래 normal 함수의 세 번째 인자)를 바꿔보며 직선 피팅이 어떻게 변하는지 관찰해보세요.
# 직선 피팅을 위한 간단한 데이터 생성
true_c0 = 2.0  # y-절편
true_c1 = 3.0  # 기울기
num_data_points = 6
x_data = np.arange(num_data_points)
noise = np.random.normal(0, 1.0, size=x_data.shape)
y_data = true_c0 + true_c1 * x_data + noise

# 최소제곱 문제를 위한 설계 행렬 A와 벡터 y 구성
A_fit = np.c_[np.ones(x_data.shape[0]), x_data]
print("설계 행렬 A의 크기:", A_fit.shape)

# QR 분해 수행
Q, R = np.linalg.qr(A_fit)
print("Q 행렬의 크기:", Q.shape)
print("R 행렬의 크기:", R.shape)

# R @ c_hat = Q.T @ y_data 풀기
QTb = Q.T @ y_data
c_hat = scipy.linalg.solve_triangular(R, QTb, lower=False)

print("\n--- 찾은 파라미터 vs 실제 파라미터 ---")
print(f"           | {'찾은 값':^7s} | {'실제 값':^7s}")
print("------------------------------------")
print(f"절편 (c₀)  | {c_hat[0]:^7.2f} | {true_c0:^7.2f}")
print(f"기울기 (c₁) | {c_hat[1]:^7.2f} | {true_c1:^7.2f}")

# 결과 시각화
plt.figure(figsize=(10, 6))
plt.scatter(x_data, y_data, label='Data Points', alpha=0.7)
plt.plot(x_data, c_hat[0] + c_hat[1] * x_data, color='red', linewidth=2, label='Fitted Line (Least Squares)')
plt.title('Straight-line Fit using QR Decomposition')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.grid(True)
plt.show()

---
# 2. 주성분 분석 (Principal Component Analysis, PCA)
---
차원 축소(Dimensionality Reduction)는 데이터의 본질적인 구조는 최대한 유지하면서, 데이터를 표현하는 변수(차원)의 수를 줄이는 기술입니다.
고차원 데이터에서 불필요한 노이즈를 제거하고, 시각화를 용이하게 하며, 머신러닝 모델의 학습 속도를 높이는 등 다양한 이점을 가집니다.

**주성분 분석(Principal Component Analysis, PCA)**은 가장 널리 사용되는 차원 축소 기법 중 하나로, 데이터의 분산(variance)이 가장 큰 방향을 새로운 좌표축(주성분)으로 설정하여 데이터를 선형 변환하는 방식입니다.

## 2.1. PCA 원리: 2D 데이터로 단계별 구현
단순한 2차원 데이터를 통해 PCA가 작동하는 각 단계를 명확하게 이해하는 데 초점을 맞춥니다.

### 데이터 생성 및 전처리
두 변수 간에 강한 양의 상관관계가 있는 2차원 데이터셋을 생성합니다. PCA는 데이터의 분산을 기반으로 하므로, 각 변수의 **평균을 0으로 맞추는 중심화(Centering)** 과정이 필수적입니다.

In [ ]:
# 2D 예제 데이터 생성
num_samples = 100
x_pca_2d = np.linspace(3, 5, num_samples)
y_pca_2d = 2 * x_pca_2d - 4 + np.random.normal(0, 0.5, num_samples)
data_2d = np.array([x_pca_2d, y_pca_2d]).T

# 데이터 전처리: 평균 중심화
mean_vec = np.mean(data_2d, axis=0)
centered_data = data_2d - mean_vec

# 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
axes[0].scatter(data_2d[:, 0], data_2d[:, 1], alpha=0.7)
axes[0].set_title("Original 2D Data (Not Centered)")
axes[0].set_xlabel("Feature 1")
axes[0].set_ylabel("Feature 2")
axes[0].axis('equal')
axes[0].grid(True)

axes[1].scatter(centered_data[:, 0], centered_data[:, 1], alpha=0.7, color='orange')
axes[1].set_title("Centered Data (Mean = 0)")
axes[1].set_xlabel("Feature 1 (centered)")
axes[1].set_ylabel("Feature 2 (centered)")
axes[1].axhline(0, color='grey', lw=1)
axes[1].axvline(0, color='grey', lw=1)
axes[1].axis('equal')
axes[1].grid(True)
plt.show()

### 주성분 계산 및 데이터 변환
PCA의 핵심은 데이터의 분산이 가장 큰 방향, 즉 **주성분(Principal Components)**을 찾는 것입니다. 이는 **공분산 행렬의 고유벡터(eigenvector)**를 계산함으로써 얻을 수 있습니다.

- **고유벡터**: 데이터의 분산이 가장 큰 방향(주성분)이 됩니다.
- **고유값**: 해당 고유벡터 방향으로 데이터가 가진 분산의 크기를 나타냅니다.

가장 중요한 주성분(PC1)을 찾은 후, 데이터를 해당 축으로 **사영(projection)**하여 차원을 축소합니다.

In [ ]:
# 공분산 행렬 계산 및 고유값 분해
# np.cov는 (변수 개수, 샘플 수) 형태의 입력을 기대하므로 전치(.T)가 필요합니다.
cov_matrix = np.cov(centered_data.T)
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix) # eigh는 대칭 행렬용

# 고유값이 큰 순서대로 정렬
sort_indices = np.argsort(eigenvalues)[::-1]
eigenvalues = eigenvalues[sort_indices]
eigenvectors = eigenvectors[:, sort_indices]
pc1 = eigenvectors[:, 0]

print("공분산 행렬:\n", np.round(cov_matrix, 2))
print("\n고유값 (분산 크기, 내림차순):\n", np.round(eigenvalues, 2))
print("\n제1 주성분 (PC1, 가장 중요한 방향):\n", np.round(pc1, 2))

# 주성분 시각화
plt.figure(figsize=(8, 6))
plt.scatter(centered_data[:, 0], centered_data[:, 1], alpha=0.7, color='orange')
plt.quiver(0, 0, pc1[0] * np.sqrt(eigenvalues[0]), pc1[1] * np.sqrt(eigenvalues[0]),
           color='red', scale_units='xy', scale=1, width=0.01, label='PC1')
plt.title("Principal Components on Centered Data")
plt.xlabel("Feature 1 (centered)")
plt.ylabel("Feature 2 (centered)")
plt.axis('equal')
plt.grid()
plt.legend()
plt.show()

# 데이터 사영 (2D -> 1D)
projected_data = centered_data @ pc1
print("원본 2D 데이터 (첫 5개):\n", np.round(data_2d[:5], 2))
print("\nPC1으로 사영된 1D 데이터 (첫 5개):\n", np.round(projected_data[:5], 2))

### 데이터 재구성
1차원으로 축소된 데이터를 다시 2차원 공간으로 되돌리는 과정입니다. 이 과정은 PCA가 정보의 손실을 감수하고 데이터의 가장 중요한 분산 방향으로 데이터를 '근사'하는 과정을 시각적으로 보여줍니다.

In [ ]:
# 1D 데이터를 다시 2D 공간으로 재구성
reconstructed_centered_data = projected_data[:, np.newaxis] @ pc1[np.newaxis, :]
# 원본 데이터 공간으로 이동 (평균을 다시 더해줌)
reconstructed_data = reconstructed_centered_data + mean_vec

# 시각화
plt.figure(figsize=(8, 8))
plt.scatter(data_2d[:, 0], data_2d[:, 1], alpha=0.5, s=100, label='Original Data')
plt.scatter(reconstructed_data[:, 0], reconstructed_data[:, 1], color='red', marker='x', s=100, label='Reconstructed Data (on PC1)')
# 원본-재구성 연결선 (오차 시각화)
for i in range(len(data_2d)):
    plt.plot([data_2d[i, 0], reconstructed_data[i, 0]], [data_2d[i, 1], reconstructed_data[i, 1]], 'k--', alpha=0.4)
plt.title("Original vs. Reconstructed Data")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.axis('equal')
plt.grid()
plt.legend()
plt.show()

## 2.2. PCA 응용: MNIST 차원 축소
고차원 데이터인 MNIST 손글씨 숫자 이미지를 PCA로 분석하여 차원 축소의 강력한 응용을 경험합니다. 각 이미지는 28x28=784개의 픽셀로 이루어진 784차원 벡터입니다.

### 데이터 준비 및 분산 설명량 확인
사용자가 선택한 숫자들에 대해 PCA를 실행하고, 각 주성분이 전체 데이터 분산의 몇 %를 설명하는지 시각화합니다.
- **Scree Plot**: 각 주성분(고유값)의 중요도를 시각적으로 보여줍니다.
- **누적 분산 설명량 그래프**: 몇 개의 주성분을 사용해야 원하는 비율(예: 95%)의 데이터 정보를 보존할 수 있는지 알려줍니다.

In [ ]:
# --- 하이퍼파라미터 설정 ---
# 실습: 분석하고 싶은 숫자들을 리스트에 포함시켜 보세요. 예: [3, 8], [0, 1, 7], list(range(10))
selected_digits = [0, 1]

if X_mnist_data is not None:
    # 선택된 숫자에 해당하는 데이터만 필터링
    filter_mask = np.isin(y_mnist_data, selected_digits)
    X_mnist = X_mnist_data[filter_mask]
    y_mnist = y_mnist_data[filter_mask]
    print(f"선택된 숫자: {selected_digits}, 데이터 크기: {X_mnist.shape}")

    # 데이터 중심화 및 PCA 실행
    X_centered_mnist = X_mnist - np.mean(X_mnist, axis=0)
    cov_matrix_mnist = np.cov(X_centered_mnist.T)
    eigenvalues_mnist, eigenvectors_mnist = np.linalg.eigh(cov_matrix_mnist)
    sort_indices_mnist = np.argsort(eigenvalues_mnist)[::-1]
    eigenvalues_mnist = eigenvalues_mnist[sort_indices_mnist]
    eigenvectors_mnist = eigenvectors_mnist[:, sort_indices_mnist]

    # 분산 설명량 계산
    explained_variance_ratio = eigenvalues_mnist / np.sum(eigenvalues_mnist)
    cumulative_explained_variance = np.cumsum(explained_variance_ratio)

    # 시각화
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    num_components_to_plot = min(20, X_centered_mnist.shape[1])
    plt.bar(range(1, num_components_to_plot + 1), explained_variance_ratio[:num_components_to_plot], alpha=0.8)
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal component index')
    plt.title('Scree Plot')
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, len(cumulative_explained_variance) + 1), cumulative_explained_variance, marker='.', linestyle='--')
    plt.axhline(y=0.95, color='r', linestyle='-', label='95% threshold')
    plt.xlabel('Number of components')
    plt.ylabel('Cumulative explained variance')
    plt.title('Cumulative Explained Variance')
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

    n_components_95 = np.argmax(cumulative_explained_variance >= 0.95) + 1
    print(f"데이터 분산의 95%를 설명하는 데 필요한 주성분 개수: {n_components_95}")

else:
    print("MNIST 데이터셋이 로드되지 않아 PCA 실행을 건너뜁니다.")

### 저차원 시각화 및 이미지 재구성
784차원의 이미지 데이터를 **가장 중요한 두 개의 주성분(PC1, PC2)**만 사용하여 2차원으로 축소하고, 그 결과를 산점도로 시각화합니다.
또한, 주성분의 개수(`k`)를 다르게 하여 원본 이미지를 재구성해보며, 적은 수의 주성분만으로도 원본 이미지의 특징이 대부분 복원되는 것을 확인합니다.

In [ ]:
if X_mnist_data is not None and X_centered_mnist.size > 0:
    # 2차원으로 사영 및 시각화
    pc1_mnist = eigenvectors_mnist[:, 0]
    pc2_mnist = eigenvectors_mnist[:, 1]
    projected_mnist = np.c_[X_centered_mnist @ pc1_mnist, X_centered_mnist @ pc2_mnist]

    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(projected_mnist[:, 0], projected_mnist[:, 1], c=y_mnist, cmap=plt.get_cmap("tab10", 10), alpha=0.6, s=10)
    plt.title("MNIST Digits Projected onto First Two PCs")
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    cbar = plt.colorbar(scatter, ticks=selected_digits)
    cbar.set_label('Digit')
    plt.grid(True)
    plt.show()

    # 이미지 재구성
    def reconstruct_image(data, eigenvectors, k):
        top_k_pcs = eigenvectors[:, :k]
        projected_data = data @ top_k_pcs
        reconstructed_data = projected_data @ top_k_pcs.T
        return reconstructed_data

    sample_indices = [np.where(y_mnist == digit)[0][0] for digit in selected_digits if np.any(y_mnist == digit)]
    
    if sample_indices:
        k_values = [1, 10, 50, 100, 300]
        num_k = len(k_values)
        num_samples_to_show = len(sample_indices)

        fig, axes = plt.subplots(num_samples_to_show, num_k + 1, figsize=(num_k * 2, num_samples_to_show * 2))
        if num_samples_to_show == 1: axes = axes.reshape(1, -1)

        for i, img_idx in enumerate(sample_indices):
            # 원본 이미지
            axes[i, 0].imshow(X_mnist[img_idx].reshape(28, 28), cmap='gray')
            axes[i, 0].set_title(f"Original ({y_mnist[img_idx]})")
            axes[i, 0].axis('off')

            # 재구성된 이미지
            for j, k in enumerate(k_values):
                if k > X_centered_mnist.shape[1]:
                    axes[i, j + 1].axis('off')
                    continue
                reconstructed = reconstruct_image(X_centered_mnist, eigenvectors_mnist, k)
                reconstructed_img = reconstructed[img_idx] + np.mean(X_mnist, axis=0)
                axes[i, j + 1].imshow(reconstructed_img.reshape(28, 28), cmap='gray')
                axes[i, j + 1].set_title(f"k={k}")
                axes[i, j + 1].axis('off')
        plt.tight_layout()
        plt.show()
else:
    print("MNIST 데이터셋이 로드되지 않아 이미지 재구성을 건너뜁니다.")